# R in Seeq Data Lab

Seeq Data Lab now provides an R environment to leverage the R language for exploring and manipulating time series and industrial process data!

## Preloaded R packages

Although Seeq Data Lab's **SPy** library is built using Python, it can be accessed in an R kernel of a Jupyter notebook with the help of the R package [reticulate](https://cran.r-project.org/web/packages/reticulate/index.html) which is a library that handles automatic R to Python conversion.

Seeq Data Lab's R environment preloads the `reticulate` library behind the scenes whenver an R notebook is opened. Also preloaded is the [IRdisplay](https://cran.r-project.org/web/packages/IRdisplay/index.html) library which is a Jupyter front end interface for rich displays in an R kernel. By preloading these R packages, you do not have to execute `library(reticulate)` and `library(IRdisplay)` in a code cell. You can confirm these packages are loaded by executing `(.packages())` to list the currently loaded R packages

In [1]:
# Show loaded R libraries/packages
print(.packages())

[1] "IRdisplay"  "reticulate" "stats"      "graphics"   "grDevices" 
[6] "utils"      "datasets"   "methods"    "base"      


## Installing R packages

Just as Seeq Data Lab's Python environment allows installation of Python packages to a local Data Lab project or globally for all Data Lab Projects (adminstrators only), R packages can also be installed locally and globally. 

To install an R package in a notebook cell, execute `install.packages('<pkg')` where \<pkg\> is the name of the R package. To install an R package in a Data Lab terminal, open a terminal and execute `R -e "install.packages('<pkg>')"`.

To install an R package globally for all Data Lab projects (administrators only), in a notebook cell execute `install.packages('<pkg>', lib=Sys.getenv('R_LIBS_GLOBAL'))` or in a terminal: `R -e "install.packages('<pkg>', lib=Sys.getenv('R_LIBS_GLOBAL'))"`. The environment variable `R_LIBS_GLOBAL` stores the actual location of globally installed R packages: `/seeq/python/global-packages/R/global-library/4.0`


The respective order of precedence for R library paths is the same as for Python: 

* local user install in a Data Lab project
* global install across all Data Lab projects
* local system (packages preinstalled by Seeq)

The order and location of R package paths can be listed:

In [ ]:
# Show order of library paths
print(.libPaths())

## Importing the SPy Module

Although **SPy** is imported by default and you are pre-authenticated just as in a Seeq Data Lab Python notebook, you can optinally import it by executing: `spy <- import('seeq.spy')`.

## SPy Commands

Unlike a Seeq Data Lab Python environment where results from SPy commands are output in HTML format automatically, in the R environment, HTML responses can only be generated if the SPy status message is contained in the `display_html` method from the `IRDisplay` package. The examples below for the various SPy comamnds include the steps to create a SPy status object, supply it as a parameter in a SPy command, and wrap the status message in the `display_html` method. 

### SPy Status

In [ ]:
# Set a SPy Status object to get back status string.
status <- spy$Status()

### SPy Login

In [ ]:
# SPy Import
spy <- import('seeq.spy')

# SPy Login
spy$login(url='http://localhost:34216', credentials_file='../credentials.key', force=FALSE, status=status)

# SPy status messages must be wrapped in display_html(status$message) to render in html
display_html(status$message)

# Confirm login by checking SPy user object is not NULL
spy$user

### SPy Search

In [ ]:
# Create Python dictinary object to store our search criteria
py_search_dict <- dict('Name'='Compressor','Path'='Example >> Cooling Tower 1 >> Area A')

# Run SPy.search query (named query parameter optional) and store into R data frame
search_results <- spy$search(query=py_search_dict, status=status)

# Display search status and results
display_html(status$message)
search_results

# Summary of returned results
str(search_results)

### SPy Pull

In [ ]:
# Pull data with SPy.pull using the search results and filtering on start and end dates, store into R data frame
pull_results <- spy$pull(search_results, start='2019-01-01', end='2019-01-07', header='Name', status=status)

# Display pull status and results
display_html(status$message)
head(pull_results,10)

# Display values of first 5 rows of 'Compressor Power' column. Use backticks to access a column with space in its name 
head(pull_results$`Compressor Power`,5)

### SPy Push

In [ ]:
# Create an R vector for each row of capsule data
operator <- c('Mark','Hedwig')
cap_tz <- 'UTC'#'America/New_York'
cap_start <- c(as.POSIXct('2019-01-01 09:00:00.000', tz=cap_tz), as.POSIXct('2019-01-02 09:00:00.000', tz=cap_tz))
cap_end <- c(as.POSIXct('2019-01-01 17:00:00.000', tz=cap_tz), as.POSIXct('2019-01-02 17:00:00.000', tz=cap_tz))

# Alternatively, create an R vector for each row of capsule data using pandas datetimes
# pd <- import('pandas')
# cap_start <- c(pd$to_datetime('2019-01-01T09:00:00.000Z'), pd$to_datetime('2019-01-02T09:00:00.000Z'))
# cap_end <- c(pd$to_datetime('2019-01-01T17:00:00.000Z'), pd$to_datetime('2019-01-02T17:00:00.000Z'))

# Create capsule data frame from the vectors
capsule_data <- data.frame(cap_start, cap_end, operator)

# Add column names to data frame
colnames(capsule_data) <- c('Capsule Start', 'Capsule End', 'Operator On Duty')
capsule_data

In [ ]:
# Create an R data frame for capsule metadata values
capsule_metadata <- data.frame('Operator Shifts3', 'Condition', '2d')

# Add column names to metadata data frame
colnames(capsule_metadata) = c('Name', 'Type', 'Maximum Duration')
capsule_metadata

In [ ]:
# Push data with SPy.push supplying capsule data, its metadata and a Status object
push_results <- spy$push(data=capsule_data, metadata=capsule_metadata, status=status)

# Show push results and status message
display_html(status$message)
push_results

### SPy Jobs

In [ ]:
# Schedule this notebook for execution
job_results <- spy$jobs$schedule('every 15 minutes', status=status)

# Show job results and status message
display_html(status$message)
job_results

In [ ]:
# Unschedule this notebook for execution
spy$jobs$unschedule(status=status)

# Show job results and status message
display_html(status$message)

### SPy Trees

In [ ]:
# Create a tree with a root asset
my_tree = spy$assets$Tree('My Tree', workbook='My Workbook', status=status)

# Show status message, results and assets as a dataframe
display_html(status$message)
my_tree
my_tree$items()

# Creating a Tree using CSV Files
my_csv_tree = spy$assets$Tree('~/SPy Documentation/spy_tree_example.csv', workbook='My Workbook', status=status)

# Show status message, results and assets as a dataframe
display_html(status$message)
my_csv_tree
my_csv_tree$items()

In [ ]:
# Insert an asset into a tree
my_tree$insert(children='Cooling Tower 1', parent='My Tree', status=status)

# Show status message, results, and dataframe
display_html(status$message)
my_tree
my_tree$items()

# Insert list of assets
my_tree$insert(children=list('Area A', 'Area B', 'Area C'), parent='Cooling Tower 1', status=status)

# Show status message, results, and dataframe
display_html(status$message)
my_tree
my_tree$items()

# Create Python dictinary object to store our search criteria
py_search_dict <- dict('Name'='Area A_Temperature','Datasource Name'='Example Data')
tree_search_results <- spy$search(py_search_dict, status=status)

# Insert the assets from the search results into the tree
my_tree$insert(children=tree_search_results, friendly_name='Temperature', parent='Area A', status=status)

# Show status message, results, and dataframe
display_html(status$message)
my_tree
my_tree$items()

In [ ]:
# Remove an asset
my_tree$remove('Area B_Compressor Stage')

# Show tree summary and visualization
my_tree$summarize()
my_tree$visualize()

# Remove an asset with children
my_tree$remove('Area B')
my_tree$visualize()

In [ ]:
# Accidentally insert "Area C_Temperature" into Area A of My Tree.
my_tree$insert(children=spy$search(dict('Name'='Area C_Temperature','Datasource Name'='Example Data')), 
               parent='Area A', status=status)
display_html(status$message)
my_tree$visualize()

In [ ]:
# Move the tree
my_tree$move(source='Area A >> Area C_Temperature', destination='Area C', status=status)
display_html(status$message)
my_tree$visualize()

In [ ]:
# Push the tree
my_tree$push(status=status)
display_html(status$message)

In [ ]:
# Push the CSV tree
my_csv_tree$push(status=status)
display_html(status$message)

## Utilities

Reticulate supports running Python code as a string using [py_run_string](https://rstudio.github.io/reticulate/reference/py_run.html) command. SPy commands can be run similarly.

In [ ]:
# Run arbitrary Python code 
py_run_string("from seeq import spy; user=spy.user;")
py$user

In [ ]:
# Print documentation on SPy or any Python object
py_help(spy$search)

In [ ]:
# list installed packages
installed.packages()

In [ ]:
# Get the installation path of an R package
find.package("reticulate")

# Get the installed version of an R package
packageVersion("reticulate")